This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [247]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [248]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "lfpr"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [249]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [250]:
# The get_series_info method may be useful
fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Consumption Change": "PCE",
    "Change Investment": "GPDI",
    "Change Government Spending": "W068RCQ027SBEA",
    "Change Imports": "IMPGS",
    "Change Exports": "EXPGS"
}

In [251]:
today = dt.date.today()

consumption_change = fred.get_series(
    series_id=series_codes["Consumption Change"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,  
    frequency='m',
).rename("Consumption Change")


investment_change = fred.get_series(
    series_id=series_codes["Change Investment"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    frequency='q'
).rename("Change Investment")

government_change = fred.get_series(
    series_id=series_codes["Change Government Spending"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    frequency='q'
).rename("Change Government Spending")

imports_change = fred.get_series(
    series_id=series_codes["Change Imports"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    frequency='q'
).rename("Change Imports")

exports_change = fred.get_series(
    series_id=series_codes["Change Exports"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    frequency='q'
).rename("Change Exports")

joined_df = pd.concat(
    [consumption_change, investment_change, government_change,imports_change, exports_change ], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)



5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [252]:
# Filter data to start from 2022
joined_df_filtered = joined_df[joined_df.index >= '2022-01-01']
# Calculate percentage change (year-over-year)
joined_df_pct = joined_df_filtered.pct_change(periods=4) * 100  # 4 quarters for year-over-year
# Drop NaN values from percentage change calculation
joined_df_pct = joined_df_pct.dropna()
# Create quarter labels with Q before year (e.g., "Q1 2022")
quarter_labels = [f"Q{(date.month-1)//3 + 1} {date.year}" for date in joined_df_pct.index]
# Create the figure
fig = go.Figure()
# Add bars for positive contributions (stacked above zero)
fig.add_trace(go.Bar(
    x=quarter_labels,
    y=joined_df_pct['Consumption Change'],
    name='Consumption',
    marker_color="#8B6F47"  # Dark tan (Consumption)
))
fig.add_trace(go.Bar(
    x=quarter_labels,
    y=joined_df_pct['Change Investment'],
    name='Investment',
    marker_color="#A67C52"  # Medium-dark tan (Investment)
))
fig.add_trace(go.Bar(
    x=quarter_labels,
    y=joined_df_pct['Change Exports'],
    name='Net Exports',
    marker_color="#C2956E"  
))
fig.add_trace(go.Bar(
    x=quarter_labels,
    y=joined_df_pct['Change Government Spending'],
    name='Government',
    marker_color="#D4B896"  # Light tan (Government)
))
# Add bars for imports - negative values to display below zero
fig.add_trace(go.Bar(
    x=quarter_labels,
    y=-joined_df_pct['Change Imports'],  # Make negative to show below axis
    name='Imports (negative)',
    marker_color="#CC82CD",  # Brick red
    base=0  # Ensure it starts from zero and goes down
))
# Update layout
fig.update_layout(
    barmode='relative',
    title={
        'text': '<span style="font-size:40px">Contributions to Annualized Change in Real GDP by Component</span><br><sup style="font-size:5px">Seasonally Adjusted, GDP Now: 2.3% </sup>',
        'x': 0.07,
        'xanchor': 'left',
        'y': 0.9,
        'yanchor': 'top'
    },
    xaxis_title='Quarter',
    yaxis_title='Percent (%) Change',
    hovermode='x unified',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="right",
        x=1,
        font=dict(size=12),
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='gray',
        borderwidth=1
    ),
    yaxis=dict(
        tickformat='.0f',
        ticksuffix='%',
        dtick=5,
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black',
        tickfont=dict(size=10)
    ),
    xaxis=dict(
        type='category',
        title_standoff=5
    ),
    height=600,
    margin=dict(l=100, t=120, b=60, r=20)
)